In [4]:
import requests
import pandas as pd

url = "https://covid-193.p.rapidapi.com/statistics"

headers = {
	"X-RapidAPI-Key": "7797e95aefmsh0d707667574f793p1d1f2cjsn91e8e58bd970",
	"X-RapidAPI-Host": "covid-193.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

# convert from json to dataframe
# pd.json_normalize ["response"]
df = pd.json_normalize(response.json()["response"])

In [5]:
df

,continent,country,population,day,time,cases.new,cases.active,cases.critical,cases.recovered,cases.1M_pop,cases.total,deaths.new,deaths.1M_pop,deaths.total,tests.1M_pop,tests.total
0,Africa,Niger,2.608366e+07,2024-04-28,2024-04-28T09:30:07+00:00,None,729.0,1.0,8890.0,381,9931,None,12,312.0,9759,254538.0
1,Africa,Saint-Helena,6.115000e+03,2024-04-28,2024-04-28T09:30:07+00:00,None,2164.0,NaN,2.0,354211,2166,None,None,NaN,None,NaN
2,South-America,Falkland-Islands,3.539000e+03,2024-04-28,2024-04-28T09:30:07+00:00,None,0.0,NaN,1930.0,545352,1930,None,None,NaN,2439107,8632.0
3,North-America,Montserrat,4.965000e+03,2024-04-28,2024-04-28T09:30:07+00:00,None,19.0,NaN,1376.0,282578,1403,None,1611,8.0,3577442,17762.0
4,None,Diamond-Princess,NaN,2024-04-28,2024-04-28T09:30:07+00:00,None,0.0,NaN,699.0,None,712,None,None,13.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Asia,India,1.406632e+09,2024-04-28,2024-04-28T09:30:02+00:00,None,NaN,NaN,NaN,32016,45035393,None,379,533570.0,665334,935879495.0
234,Europe,France,6.558452e+07,2024-04-28,2024-04-28T09:30:02+00:00,None,0.0,NaN,39970918.0,612013,40138560,None,2556,167642.0,4139547,271490188.0
235,Europe,Germany,8.388360e+07,2024-04-28,2024-04-28T09:30:02+00:00,None,405368.0,NaN,38240600.0,462891,38828995,None,2182,183027.0,1458359,122332384.0
236,South-America,Brazil,2.153536e+08,2024-04-28,2024-04-28T09:30:02+00:00,None,1783377.0,NaN,36249161.0,179908,38743918,None,3303,711380.0,296146,63776166.0


In [6]:
import dotenv
dotenv.load_dotenv()

False

In [10]:
import os
import pandas as pd
import requests

class RapidAPIManager:
    def __init__(self, api_key, api_host, url):
        self.__api_key = api_key
        self.__api_host = api_host
        self.url = url
        self.__headers = {
            "X-RapidAPI-Key": self.__api_key,
            "X-RapidAPI-Host": self.__api_host
    }
        
    # self for access to object's state
    def get_data(self):
        response = requests.get(self.url, headers=self.__headers)
        if response.status_code == 200:
            return pd.json_normalize(response.json()["response"])
        else:
            raise Exception(f"Failed to retrieve data, status code: {response.status_code}")
    
    def save_to_csv(self, data, file_name, path):
        try:
            if not os.path.exists(path):
                os.makedirs(path)

            # full path
            full_file_path =  os.path.join(path, file_name)

            # convert to csv
            data.to_csv(full_file_path, index=False)
            print(f"File '{file_name}' saved to disk in '{path}' folder")  

        except Exception as e:
            print(e)

In [12]:
# # create instance of class with provided api & host
# covid_stats = RapidAPIManager(
#     os.getenv("RAPID_API_KEY"),
#     os.getenv("RAPID_API_HOST"),
#     os.getenv("RAPID_API_URL"))

# # use instance to get data & store in df
# df = covid_stats.get_data()
# df

In [14]:
covid_stats.save_to_csv(
    data=df,
    file_name="covid19_raw_data.csv",
    path="data/")

File 'covid19_raw_data.csv' saved to disk in 'data/' folder


In [7]:
import boto3

class AWS_S3Manager:

    def __init__(self, bucket_name, aws_access_key_id, aws_secret_access_key, region_name):
        self.bucket_name = bucket_name
        self.s3_client = boto3.client(
            service_name='s3',
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
        )

    def upload_file(self, local_filename, s3_key):
        
        try:

            self.s3_client.upload_file(local_filename, self.bucket_name, s3_key)
            print(f"File '{local_filename}' uploaded to S3 bucket as '{s3_key}'.")

        except Exception as e:
            print(e)
    def download_file(self, s3_key, local_filename, target_directory="downloads"):
        
        try:

            if not os.path.exists(target_directory):
                os.makedirs(target_directory)# create the target directory if it doesn't exist

            # Construct the full local path
            local_path = os.path.join(target_directory, local_filename)

            # Download the file
            self.s3_client.download_file(self.bucket_name, s3_key, local_path)
            print(f"File '{s3_key}' downloaded from S3 bucket and saved as '{local_path}'.")

        except Exception as e:
            print(e)

    def read_csv_from_s3(self, s3_key):
        try:
        
            obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=s3_key)
            df = pd.read_csv(obj['Body'])
            return df
        
        except Exception as e:
            print(e)

In [8]:
import os
s3_managet = AWS_S3Manager(
    bucket_name=os.getenv("S3_BUCKET_NAME"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

In [9]:
s3_managet.upload_file(
    local_filename="data/covid19_raw_data.csv",
    s3_key="ym_covid19_raw_data.csv"
)

expected string or bytes-like object, got 'NoneType'


In [10]:
data = s3_managet.read_csv_from_s3(s3_key="ym_covid19_raw_data.csv")

expected string or bytes-like object, got 'NoneType'


In [11]:
data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [12]:
class DataTransformation:
    
    def __init__(self):
        pass
    
    def data_transformation(self,data):
        
        try:
            # define the original and new column names
                original_columns = ['continent', 'country', 'population', 'day', 'time', 'cases.new',
                    'cases.active', 'cases.critical', 'cases.recovered', 'cases.1M_pop',
                    'cases.total', 'deaths.new', 'deaths.1M_pop', 'deaths.total',
                    'tests.1M_pop', 'tests.total']

                new_columns = ['continent', 'country', 'population', 'day', 'time', 'cases_new',
                    'cases_active', 'cases_critical', 'cases_recovered', 'cases_1M_pop',
                    'cases_total', 'deaths_new', 'deaths_1M_pop', 'deaths_total',
                    'tests_1M_pop', 'tests_total']
                
                rename_dict = {original: new for original, new in zip(original_columns, new_columns)}

                # rename the columns
                data.rename(columns=rename_dict, inplace=True)
                
                #we have to drop unwanted features
                data.drop(columns=["time", "cases_new", "deaths_new","cases_critical"], inplace=True)
                
                # replace nan values with median for numeric values and mode fro categorical values
                data["continent"]  = data["continent"].fillna(st.mode(data["continent"]))
                data["population"] = data["population"].fillna(np.nanmedian(data["population"]))
                data["cases_active"] = data["cases_active"].fillna(np.nanmedian(data["cases_active"]))
                data["cases_recovered"] = data["cases_recovered"].fillna(np.nanmedian(data["cases_recovered"]))
                data["cases_1M_pop"] = data["cases_1M_pop"].fillna(np.nanmedian(data["cases_1M_pop"]))
                data["deaths_1M_pop"] = data["deaths_1M_pop"].fillna(np.nanmedian(data["deaths_1M_pop"]))
                data["deaths_total"] = data["deaths_total"].fillna(np.nanmedian(data["deaths_total"]))
                data["tests_total"] = data["tests_total"].fillna(np.nanmedian(data["tests_total"]))
                
                #convert in to date time
                data['day'] = pd.to_datetime(data['day'])
                
                return data
                
        except Exception as e:
            print(e)
            
            
    def save_to_csv(self, data, file_name, path):
        try:
            # create the folder if it does not exist
            if not os.path.exists(path):
                os.makedirs(path)

            # specify the full file path
            full_file_path = os.path.join(path, file_name)

            # ssave the data to a CSV file....
            data.to_csv(full_file_path, index=False)
            print(f"File '{file_name}' saved to disk in the '{path}' folder.")
        
        except Exception as e:
            print(e)

In [13]:
dt = DataTransformation()
data = dt.data_transformation(data=data)

'NoneType' object has no attribute 'rename'


In [14]:
dt.save_to_csv(
    data=data,
    file_name="transform_covid_data.csv",
    path="data/"
)

'NoneType' object has no attribute 'to_csv'


In [15]:
s3_managet.upload_file(
    local_filename="data/transform_covid_data.csv",
    s3_key="ym_transform_covid_data.csv"
)

expected string or bytes-like object, got 'NoneType'


In [16]:
import pymysql 
import sqlalchemy
from sqlalchemy import create_engine

class DatabaseManager:

    def __init__(self, connection_string):
        try:
            pymysql.install_as_MySQLdb()  # install pymysql as MySQLdb for compatibility
            self.db_engine = create_engine(connection_string)
            self.conn = self.db_engine.connect()

        except Exception as e:
            raise CustomException(e,sys)
            
            
    def import_data_to_table(self,data,table_name):
        
        '''
        arguments: data, table_name
        
        This method imports data to a table in the database.
        '''
        try:
            
            data.to_sql(table_name, self.conn, if_exists='append', index=False) 
            print(f"Data imported to table '{table_name}'.")
            
        except Exception as e:
            print(e)

    def import_csv_to_table(self, file_path, table_name):
        
        '''
        arguments: file_path, table_name
        
        Importing  CSV Data to table in data base
        '''
        try:
            
            df = pd.read_csv(file_path, encoding='latin1', sep=",")
            df.to_sql(table_name, con=self.conn, if_exists='replace')

        except Exception as e:
            print(e)
        

    def close_connection(self):
        self.conn.close()

ModuleNotFoundError: No module named 'pymysql'

In [17]:
#creating object of DatabaseManager
db_manager = DatabaseManager(
    connection_string=os.getenv("DB_ALCHEMY_CONNECTION_STRING")
        )
        
db_manager.import_data_to_table(
        data=data,
        table_name="ym_covid19_data",)
        
db_manager.close_connection()

NameError: name 'DatabaseManager' is not defined